## Import Modules

In [1]:
import os
import pandas as pd
import pathlib
import json
import numpy as np
import json
import struct
import subprocess
import torch
import re
import unicodedata
import nltk
import spacy

from datasets import load_dataset
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline,
    TextDataset,
    EvalPrediction,
    DataCollatorWithPadding,
    GenerationConfig,
    BitsAndBytesConfig,
    MarianMTModel,
    MarianTokenizer,
    T5Tokenizer,
    T5ForConditionalGeneration
)

from nltk.corpus import stopwords
from contractions import fix
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textblob import TextBlob
from spellchecker import SpellChecker

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available.")

d:\Python\LLM_Environment\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



GPU is available!


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
MAIN_PATH = str(pathlib.Path().resolve())
DATASET_PATH = MAIN_PATH + '\\datasets'
MODEL_PATH = MAIN_PATH + '\\models'
MODELS = 'D:\\AI\\LLM\\models'

## Set Up Model Path

In [4]:
models = os.listdir(MODEL_PATH)
models

['bart-large-cnn',
 'bert-base-cased',
 'bert-base-multilingual-cased',
 'bert-base-uncased',
 'bert-large-cased',
 'bert-large-uncased',
 'flan-t5-base',
 'flan-t5-large',
 'flan-t5-small',
 'gpt2',
 'gpt2-large',
 'gpt2-medium',
 'opus-mt-en-id',
 'opus-mt-id-en',
 't5-base',
 't5-large']

## Select Dataset

In [5]:
counter = 0
n = 20

path_list = []

for dirpath, dirnames, filenames in os.walk(DATASET_PATH):
    for filename in filenames:
        print(dirpath + '/' + filename)
        path_list.append(dirpath + '/' + filename)
    counter += 1
    if counter >= n:
        break

D:\Python\LLM_Environment\datasets/Australian_Legal_Corpus.jsonl
D:\Python\LLM_Environment\datasets/cached_lm_GPT2Tokenizer_128_Shakespeare_Dataset.txt
D:\Python\LLM_Environment\datasets/Customer.csv
D:\Python\LLM_Environment\datasets/Html.csv
D:\Python\LLM_Environment\datasets/Hukum_Online_Klinik.xlsx
D:\Python\LLM_Environment\datasets/Recipes.csv
D:\Python\LLM_Environment\datasets/Recipes_1000.csv
D:\Python\LLM_Environment\datasets/Shakespeare_Dataset.txt
D:\Python\LLM_Environment\datasets/Taylor_Swift_Lyrics.csv
D:\Python\LLM_Environment\datasets/Twitter.csv


In [6]:
file_format = '.xlsx'

file_paths = []

for root, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        if file.endswith(file_format):
            file_paths.append(os.path.join(root, file))
        if len(file_paths) >= n:
            break
    if len(file_paths) >= n:
        break
file_paths[0]

'D:\\Python\\LLM_Environment\\datasets\\Hukum_Online_Klinik.xlsx'

In [7]:
file_path = file_paths[0]
file_path

'D:\\Python\\LLM_Environment\\datasets\\Hukum_Online_Klinik.xlsx'

## Read Dataset

### Read as CSV

In [8]:
try:
    df = pd.read_csv(file_path, encoding = 'ISO-8859-1')#, sep = ';')
    print("DataFrame shape:", df.shape)
    display(df.head())
except:
    pass

### Read as Excel

In [9]:
try:
    df = pd.read_excel(file_path)
    print("DataFrame shape:", df.shape)
    display(df.head())
except:
    pass

DataFrame shape: (8371, 5)


,Title,Link,Question,Summary,Answer
0,PengertianOvermachtdanUltimum Remediumdalam Hu...,https://www.hukumonline.com/klinik/a/pengertia...,Bagaimana ketentuanovermachtdalam hukum pidana...,Daya paksa atauovermachtdalam hukum pidana mer...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...
1,Ini 6 Perbedaan CV dan PT yang Wajib Diketahui,https://www.hukumonline.com/klinik/a/perbedaan...,Saya punya tiga pertanyaan. Apa perbedaan PT d...,PT adalah singkatan dari perseroan terbatas ya...,Terima kasih atas pertanyaan Anda.Artikel di b...
2,"KDRT Hingga Meninggal, Penganiayaan atau Pembu...",https://www.hukumonline.com/klinik/a/kdrt-hing...,"Belakangan ini, viral kasus seorang suami di S...",Pasal KDRT atau kekerasan dalam rumah tangga y...,Terima kasih atas pertanyaan Anda.Artikel di b...
3,7 Cara Memperoleh Wilayah dalam Hukum Internas...,https://www.hukumonline.com/klinik/a/7-cara-me...,Konflik Palestina-Israel masih berlarut-larut ...,"Pada dasarnya, terdapat 7 cara memperoleh wila...",Terima kasih atas pertanyaan Anda.Seluruh info...
4,"Pacar Tidak Mau Bertanggung Jawab, Kapan Maksi...",https://www.hukumonline.com/klinik/a/pacar-tid...,Saya ingin menanyakan bila laki-laki dan perem...,Jika terjadi persetubuhan antara sepasang keka...,Terima kasih atas pertanyaan Anda.Artikel di b...


### Read as JSON

In [10]:
counter = 0
row = 20

try:
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            print(data)
            counter += 1

            if counter >= row:
                break
except:
    pass

In [11]:
n = 50
chunk_size = 1000
total_rows_processed = 0

data_frames = []

try:
    with open(file_path, 'r') as file:
        counter = 0

        while counter < n:
            data_list = []

            for _ in range(chunk_size):
                line = file.readline()
                if not line:
                    break

                data = json.loads(line)
                data_list.append(data)

            if data_list:
                df_chunk = pd.DataFrame(data_list)
                data_frames.append(df_chunk)

                print(f'Processed chunk with {len(df_chunk)} rows.')
                total_rows_processed += len(df_chunk)
                counter += 1

            else:
                break

    print(f'Total rows processed: {total_rows_processed}')

    # (Optional) Concatenate all DataFrames if needed
    if data_frames:  # Check if there are any DataFrames to concatenate
        df = pd.concat(data_frames, ignore_index = True)
        print('Concatenated DataFrame shape:', df.shape)
        display(df.head())
except:
    pass

### Read as Parquet

In [12]:
try:
    df = pd.read_parquet(file_path)
    print("DataFrame shape:", df.shape)
    display(df.head())
except:
    pass

## 04. Translation

In [13]:
dfa = df.iloc[:10]
dfa.shape

(10, 5)

In [14]:
models

['bart-large-cnn',
 'bert-base-cased',
 'bert-base-multilingual-cased',
 'bert-base-uncased',
 'bert-large-cased',
 'bert-large-uncased',
 'flan-t5-base',
 'flan-t5-large',
 'flan-t5-small',
 'gpt2',
 'gpt2-large',
 'gpt2-medium',
 'opus-mt-en-id',
 'opus-mt-id-en',
 't5-base',
 't5-large']

In [15]:
model_path = MODEL_PATH + '\\' + models[8]
model_path

'D:\\Python\\LLM_Environment\\models\\flan-t5-small'

In [16]:
model = T5ForConditionalGeneration.from_pretrained(model_path, torch_dtype = torch.float16).to('cuda')
tokenizer = T5Tokenizer.from_pretrained(model_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Count Token Length

In [17]:
text = dfa.iloc[0][4]
text

C:\Users\422in\AppData\Local\Temp\ipykernel_52124\4293223326.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = dfa.iloc[0][4]


'KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012Terima kasih atas pertanyaan Anda.Artikel di bawah ini adalah pemutakhiran dari artikel dengan judulTentang\xa0Overmacht\xa0dan Hukum Pidana sebagai\xa0Ultimum Remediumyang dibuat olehAnandito Utomo, S.H.dan dipublikasikan pada 19 Agustus 2016.Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya TerjangkauMulai DariRp. 149.000Lihat Semua KelasArtikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023.Seluruh informasi hukum yang ada di Klinik hukumonline.com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihatPernyataan Penyangkalanselengkapnya). Untuk mendapatkan nasihat hukum spesifik terhadap kasus

In [18]:
def count_tokens(text):
    tokens = tokenizer(text, return_tensors = "pt", truncation = False).to('cuda')
    return len(tokens['input_ids'][0])

token_length = count_tokens(text)
print(f"Token length: {token_length}")

Token indices sequence length is longer than the specified maximum sequence length for this model (6648 > 512). Running this sequence through the model will result in indexing errors


Token length: 6648


### Clean Text

In [19]:
def clean_text(text):
    text = text.replace('\xa0', ' ').strip()
    text = text.replace('.', '. ')
    text = text.replace('(', ' (')
    text = text.replace(')', ') ')
    text = text.replace('[', ' [')
    text = text.replace(']', '] ')
    text = text.replace(',', ', ')
    text = text.replace(':', ': ')
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    #text = re.sub(r'([A-Za-z])([A-Z]{2,})', r'\1 \2', text)
    text = re.sub(r'(\d+)([A-Za-z])', r'\1 \2', text)
    text = re.sub(r'(\d{1,2})([A-Z][a-z]+),(\s*\d{4})', r'\1 \2 \3', text)
    text = re.sub(r'Mulai DariRp\..*', '', text)
    text = re.sub(r'\d+\.\d+', '', text)

    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    
    return cleaned_text

cleaned_text = clean_text(text)
print(cleaned_text)

KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Penyangkalanselengkapnya) . Untuk mendapatkan nasihat hukum spesifik terha

### Translate

In [20]:
text = cleaned_text

### 3rd method

In [21]:
torch.cuda.empty_cache()

In [22]:
os.listdir(MODELS)

['config.yaml',
 'gemma-2-2b',
 'Llama-3-8B-GPTQ-4-Bit',
 'Llama-3-8B-Instruct-GPTQ-4-Bit',
 'Llama-3.2-11B-Vision-Instruct-bnb-4bit',
 'Llama-3.2-1B-Instruct',
 'Llama-3.2-3B-Instruct',
 'Meta-Llama-3.1-8B-Instruct-GPTQ-INT4',
 'Phi-3-mini-128k-instruct',
 'Phi-3-mini-128k-instruct-onnx',
 'Phi-3-mini-4k-instruct-q4.gguf',
 'Phi-3.5-mini-instruct',
 'Phi-3.5-vision-instruct',
 'place-your-models-here.txt',
 'Qwen2.5-0.5B',
 'Qwen2.5-0.5B-Instruct',
 'Qwen2.5-1.5B',
 'Qwen2.5-1.5B-Instruct',
 'Qwen2.5-3B',
 'Qwen2.5-3B-Instruct',
 'Qwen2.5-7B-Instruct-GPTQ-Int4',
 'Qwen2.5-Coder-1.5B',
 'Qwen2.5-Coder-1.5B-Instruct',
 'Qwen2.5-Coder-7B-Instruct-GPTQ-Int4',
 'Qwen2.5-Math-1.5B',
 'Qwen2.5-Math-1.5B-Instruct']

In [23]:
model_path = MODELS + '\\' + os.listdir(MODELS)[6]
model_path

'D:\\AI\\LLM\\models\\Llama-3.2-3B-Instruct'

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.float16,
    trust_remote_code = True
).to(device) #'''

translator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [25]:
def split_text_into_chunks(text, max_tokens=8000):
    tokens = tokenizer(text, return_tensors='pt', truncation=False)['input_ids'][0]
    token_length = len(tokens)

    if token_length <= max_tokens:
        return [text]

    chunks = []
    for i in range(0, token_length, max_tokens):
        chunk_tokens = tokens[i:min(i + max_tokens, token_length)]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)

    return chunks

def translate_large_text(text):
    chunks = split_text_into_chunks(text)
    formatted_chunks = [f"Translate the following Indonesian text to English: {chunk}" for chunk in chunks]
    translated_chunks = [translator(chunk, max_length=8000)[0]['generated_text'] for chunk in formatted_chunks]
    return ' '.join(translated_chunks)


translated_text = translate_large_text(text)
print(translated_text)


d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Translate the following Indonesian text to English: KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Penyangkalanselengkapn

In [26]:
token_length = count_tokens(translated_text)
print(f"Token length: {token_length}")

Token length: 8000


In [27]:
translator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

def split_text_into_chunks(text, max_tokens=8000):
    tokens = tokenizer(text, return_tensors='pt', truncation=False)['input_ids'][0]
    token_length = len(tokens)

    if token_length <= max_tokens:
        return [text]

    chunks = []
    for i in range(0, token_length, max_tokens):
        chunk_tokens = tokens[i:min(i + max_tokens, token_length)]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)

    return chunks

def translate_large_text(text):
    chunks = split_text_into_chunks(text)
    formatted_chunks = [f"Translate this Indonesian text to English: {chunk}" for chunk in chunks]

    translated_chunks = []
    for chunk in formatted_chunks:
        translated = translator(chunk, max_length=8000, do_sample=False)[0]['generated_text']
        translated_chunks.append(translated.strip())
    
    return ' '.join(translated_chunks)

translated_text = translate_large_text(text)
print(translated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Translate this Indonesian text to English: KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Penyangkalanselengkapnya) . Unt

In [28]:
token_length = count_tokens(translated_text)
print(f"Token length: {token_length}")

Token length: 8000


### Question Generation

In [29]:
question_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

def generate_questions(text):
    prompt = f"### Human:Generate questions based on the following text:{text}\n### Assistant:"
    generated_text = question_generator(prompt, max_new_tokens=800, do_sample=False)[0]['generated_text']
    questions = [q.strip() for q in generated_text.split('\n') if q.strip()]
    return questions

questions = generate_questions(text)
print("Generated Questions:")
for question in questions:
    print(question)
    token_length = count_tokens(question)
    print(f"Token length: {token_length}")

Generated Questions:
### Human:Generate questions based on the following text:KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyat

In [30]:
def assistant(prompt, task_type='translate'):
    if task_type == 'translate':
        prompt = f"### Human:Translate this Indonesian text to English: {prompt}\n### Assistant:"
    elif task_type == 'question':
        prompt = f"### Human:Generate questions based on the following text:{prompt}\n### Assistant:"
    else:
        raise ValueError("task_type must be either 'translate' or 'question'")
    
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

    generation_config = GenerationConfig(
        do_sample=True,
        top_k=1,
        temperature=0.1,
        max_new_tokens=600,
        pad_token_id=tokenizer.eos_token_id
    )

    outputs = model.generate(**inputs, generation_config=generation_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

translated_text = assistant(text, task_type='translate')
print("Translated Text:", translated_text)
token_length = count_tokens(translated_text)
print(f"Token length: {token_length}")

questions = assistant(text, task_type='question')
print("Generated Questions:", questions)
token_length = count_tokens(questions)
print(f"Token length: {token_length}")

Translated Text: ### Human:Translate this Indonesian text to English: KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Peny

In [31]:
def assistant(prompt):
    prompt = f"### Human:Generate 50 questions based on the following text:{prompt}\n### Assistant:"
    inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

    generation_config = GenerationConfig(
        do_sample = True,
        top_k = 1,
        temperature = 0.1,
        max_new_tokens = 1500,
        pad_token_id = tokenizer.eos_token_id
    )

    outputs = model.generate(**inputs, generation_config = generation_config)
    return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [32]:
assistant(text)

### Human:Generate 50 questions based on the following text:KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Penyangkalanse

### Instruction Generation

In [33]:
def assistant(prompt):
    prompt = f"### Human:Generate 50 instructions based on the following text:{prompt}\n### Assistant:"
    inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

    generation_config = GenerationConfig(
        do_sample = True,
        top_k = 1,
        temperature = 0.1,
        max_new_tokens = 1500,
        pad_token_id = tokenizer.eos_token_id
    )

    outputs = model.generate(**inputs, generation_config = generation_config)
    return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [34]:
assistant(text)

### Human:Generate 50 instructions based on the following text:KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Penyangkala

### Prompt Generation

In [35]:
def assistant(prompt):
    prompt = f"### Human:Generate 50 prompts based on the following text:{prompt}\n### Assistant:"
    inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

    generation_config = GenerationConfig(
        do_sample = True,
        top_k = 1,
        temperature = 0.1,
        max_new_tokens = 1500,
        pad_token_id = tokenizer.eos_token_id
    )

    outputs = model.generate(**inputs, generation_config = generation_config)
    return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [36]:
assistant(text)

### Human:Generate 50 prompts based on the following text:KLINIK TERKAITDaya Paksa dan Pembelaan Terpaksa sebagai Alasan Penghapus Pidana11 Jan, 2023 Arti Ultimum Remedium sebagai Sanksi Pamungkas15 Jul, 2022 Litigasi dan Alternatif Penyelesaian Sengketa di Luar Pengadilan10 Jul, 2020 Bisakah Dipidana Karena Menembak Anjing yang Menyerang Orang Lain?22 Okt, 2012 Terima kasih atas pertanyaan Anda. Artikel di bawah ini adalah pemutakhiran dari artikel dengan judul Tentang Overmacht dan Hukum Pidana sebagai Ultimum Remediumyang dibuat oleh Anandito Utomo, S. H. dan dipublikasikan pada 19 Agustus 2016. Belajar Hukum Secara Online dari Pengajar Berkompeten Dengan Biaya Terjangkau Mulai Dari Rp. 149. 000 Lihat Semua Kelas Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023. Seluruh informasi hukum yang ada di Klinik hukumonline. com disiapkan semata – mata untuk tujuan pendidikan dan bersifat umum (lihat Pernyataan Penyangkalansele